># Analyse Spatio-Temporelle Conjointe

>## Variance expliquée par l'espace et par le temps
>### Pourquoi compare t-on la variance par l'espace et la variance par le temps ?
L'analyse spatio-temporelle d'un champ de température de surface sert d'abord à déterminer si la variabilité qu'on observe est majoritairement due à des contrastes spatiaux persistants ou à des fluctuations temporelles globales.
>### Comment calculer la variance par l'espace ?
On calcule d'abord pour chaque point la moyenne temporelle :
$$
\overline{SST}(x,y) = \frac{1}{T}\sum_{t=1}^{T}SST(x,y,t)
$$
Cette moyenne une fois représenté sur une carte permet de voir l'évolution "moyenne" du domaine (variabilité interannuelle, oscillations, éventuelle tendance).
Puis on calcule ensuite l'amplitude des constrastes spatiaux grâce à la variance spatiale de la carte :
$$
\mathrm{Var}_{espace}=\mathrm{Var}_{x,y}\overline{\mathrm{SST}}(x,y)
$$

>### Comment calculer la variance par le temps ?
On calcule la moyenne spatiale sur notre domaine à chaque instant t :
$$
\overline{SST}(t)=\frac{1}{N}\sum_{x,y}SST(x,y,t)
$$
Cette série temporelle représente l'évolution moyenne. Puis on mesure la variabilité temporelle globale avec la formule :
$$
Var_{temps}=Var_{t}\overline{SST(t)}
$$
>### Comment interpréter notre comparaison ?
On peut introduire un ratio R tel que :
$$
R = \frac{Var_{espace}}{Var_{temps}}
$$
Ainsi :
- Si on obtient R >> 1 alors notre variabilité sera principalement causée par l'espace, donc les différences géographiques dominent les fluctuations temporelles globales
- Si on obtient R << 1 alors la variabilité sera causée par le temps signifiant que le domaine évolue de manière homogène avec un signal temporel fort
- Si on obtient R = 1 les deux seront comparables ce qui justifiera d'analyser à la fois les structures spatiales et la dynamique temporelle.

In [ ]:
import numpy as np 
import xarray as xr 

datasetBrut = xr.open_dataset("../../data/processed/sstCOPERNICUS20102019Processed.nc")
sstBrut = datasetBrut["analysed_sst"]

datasetDesaisonnalise = xr.open_dataset("../../data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")
sstDesaisonnalise = datasetDesaisonnalise["analysed_sst"]

## For the Processed data :
# Temporal average and then Spatial variability
sstMeanTimeP = sstBrut.mean(dim="time")
varSpaceP = sstMeanTimeP.var(dim=("latitude","longitude"))

# Spatial average and then Temporal variability
sstMeanSpaceP = sstBrut.mean(dim=("latitude","longitude"))
varTimeP = sstMeanSpaceP.var(dim="time")

# Ratio
R1 = varSpaceP / varTimeP

print("La variance spatiale (brute) est : ",float(varSpaceP.values))
print("La variance temporelle (brute) est : ", float(varTimeP.values))
print("R1 = ", R1.values)



# Temporal average and then Spatial variability
sstMeanTime = sstDesaisonnalise.mean(dim="time")
varSpace = sstMeanTime.var(dim=("latitude","longitude"))

# Spatial average and then Temporal variability
sstMeanSpace = sstDesaisonnalise.mean(dim=("latitude","longitude"))
varTime = sstMeanSpace.var(dim="time")

# Ratio
R2 = varSpace / varTime

print("La variance spatiale est : ",float(varSpace.values))
print("La variance temporelle est : ", float(varTime.values))
print("R2 = ", float(R2.values))

La variance spatiale (brute) est :  0.048274967819452286
La variance temporelle (brute) est :  10.05994987487793
R1 =  0.0047987285
La variance spatiale est :  2.145392153860004e-12
La variance temporelle est :  0.6674704551696777
R2 =  3.214212966964891e-12


>### Comment interpréter nos résultats ?
Lorsqu'on utilise notre jeu de données brute, la variance par le temps de la moyenne spatiale est largement supérieure à la variance spatiale de la moyenne temporelle ce qui signfie que les fluctuations temporelles sont plus importantes que les fluctuations spatiales à l'échelle globale notamment le cycle saisonnier. A l'inverse lorsqu'on utilise les données désaisonnalisées la variance spatiale devient presque totalement nulle ce qui confirme que les anomalies ne sont plus impactées par la structure spatiale mais par la dynamique temporelle.


>## Covariance Spatiale
>### Pourquoi réalise t-on la covariance spatiale ?
Dans la partie précédente nous avons prouvés que la variance spatiale moyenne est faible cependant cela ne signifie pas que notre espace n'est pas structuré. On cherche donc à répondre à la question :
Est-ce que des points proches ont tendance à fluctuer ensemble dans le temps ?
Ce qui est exactement le but de la covariance spatiale.
>### Comment calcule t-on la covariance ?
On part des données désaisonnalisées puisqu'on s'intéresse à la dynamique du système puis on utilise la formule suivante :
$$
Cov_{ij}=\langle SST_{i}(t)SST_{j}(t) \rangle
$$
De plus puisque nos anomalies sont centrées nous n'avons pas besoin de retirer la moyenne (nulle) et ainsi la covariance mesure directement la covariabilité.